In [ ]:
import json
import os
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils

In [ ]:
rds = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

In [ ]:
query = """
SELECT
  *
FROM prod.biomass_computations
WHERE
  group_id='151-ENGALL-1455'
  and captured_at BETWEEN '2020-10-01' and '2020-10-16'
"""

In [ ]:
df = rds.extract_from_database(query)

In [ ]:
df.camera_metadata.iloc[0]

In [ ]:
query = """
    SELECT
      *
    FROM prod.biomass_computations
    WHERE
      group_id='151'
      and captured_at BETWEEN '2020-10-01' and '2020-10-16'
"""

In [ ]:
odf = rds.extract_from_database(query)

In [ ]:
odf.camera_metadata.iloc[0]

<h1> Perform un- and re-rectification

In [ ]:
from typing import Tuple
import numpy as np
import cv2

s3_access_utils = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))

IMAGE_WIDTH = 4096
IMAGE_HEIGHT = 3000


def load_params(params):
    print("Loading params...")
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)

    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)

    print("Params loaded.")
    return left_maps, right_maps, cameraMatrix1, distCoeffs1, R1, P1


def rectify_keypoints(annotation: dict, stereo_parameters: dict) -> dict:
    """Generated rectified key-points from unrectified key-point input and stereo parameters."""
    
    left_maps, right_maps, cameraMatrix1, distCoeffs1, R1, P1 = get_camera_parameters(stereo_parameters)
    
    rectified_ann = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in annotation[side]:
            x, y = item['xFrame'], item['yFrame']
            maps = left_maps_m if side == 'leftCrop' else right_maps_m
            x_new, y_new = cv2.undistortPoints(np.array([[[x, y]]]).astype(float), 
                                cameraMatrix1, distCoeffs1, R=R1, P=P1)[0][0]
            x_new, y_new = int(round(x_new)), int(round(y_new))
            rectified_ann[side].append({
                'keypointType': item['keypointType'],
                'xFrame': x_new,
                'yFrame': y_new
            })
            
    return rectified_ann



In [ ]:
o_params_url = 'http://aquabyte-stereo-parameters.s3.eu-west-1.amazonaws.com/L40034372_R40034364/2020-06-12T13:17:24.565125000Z_L40034372_R40034364_stereo-parameters.json'
n_params_url = 'http://aquabyte-stereo-parameters.s3.eu-west-1.amazonaws.com/L40034372_R40034364/2020-11-06T13:36:51.161503000Z_L40034372_R40034364_stereo-parameters.json'
o_params_f, _, _ = s3_access_utils.download_from_url(o_params_url)
n_params_f, _, _ = s3_access_utils.download_from_url(n_params_url)
o_params = json.load(open(o_params_f))
n_params = json.load(open(n_params_f))
left_maps_o, right_maps_o, cameraMatrix1_o, distCoeffs1_o, R1_o, P1_o = load_params(o_params)
left_maps_n, right_maps_n, cameraMatrix1_n, distCoeffs1_n, R1_n, P1_n = load_params(n_params)


In [ ]:
ann_o_op_n_list = []
for idx, row in df.iterrows():
    ann_o = row.annotation
    
    # un-rectify with matlab params, re-rectify with circular params
    ann_o_op_n = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann_o[side]:
            bp = item['keypointType']
            x, y = item['xFrame'], item['yFrame']
            maps = left_maps_o if side == 'leftCrop' else right_maps_o
            x_new, y_new = cv2.undistortPoints(np.array([[maps[0][y, x]]]).astype(float), 
                                cameraMatrix1_n, distCoeffs1_n, R=R1_n, P=P1_n)[0][0]
            x_new, y_new = int(round(x_new)), int(round(y_new))
            ann_o_op_n[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new
            })
            
    ann_o_op_n_list.append(ann_o_op_n)
            
    

In [ ]:
df['ann_o_op_n'] = ann_o_op_n_list